In [16]:
import numpy as np
import pyopenms
import matplotlib.pyplot as plt

<h1>Introduction</h1>
PyOpenMS is a set of Python bindings for the OpenMS C++ library. The included functions are powerful but can be daunting compared to native Python libraries. This SOP will serve as a guide to PyOpenMS concepts and include examples of how to do some routine data processing. 

<h1>File Processing</h1>

We'll start by opening an MzML file and viewing its contents. The first step is to import the MzML module as well as the classes that describe experiments and spectra.

In [17]:
from pyopenms import MzMLFile, MSExperiment, MSSpectrum

Now we'll instantiate an MzMLFile object and an MSExperiment obejct, and load our file into the experiment. In this example we'll open the file P3_5+_3.mzML.

In [18]:
file = MzMLFile()  # Instantiate an MzMLFile object.
exp = MSExperiment()  # Instantiate an empty MSExperiment object.

file.load("P3_5+_3.mzML", exp)  # Load the data from P3_5+_3.mzML into the newly instantiated experiment.

In OpenMS terminology, an "experiment" is a collection of spectra, such as an LC-MS dataset or an infusion experiment with a lot of averages. The experiment, called exp here, now contains the spectral data from P3_5+\_3.mzml. Individual spectra can be accessed by their index within the experiment like this: 

In [19]:
spectrum = exp[0]  # Access the first element (i.e. scan) of the experiment.

print(spectrum)

As you can see, accessing the first element of our experiment returns a MSSpectrum object. This object contains a lot of information about the spectrum. If you're planning on doing any sort of spectral processing, the most important method is <code>get_peaks()</code>, which returns a tuple of arrays containing the raw mass and intensity values for each point in the spectrum:

In [20]:
peaks = spectrum.get_peaks()

print(peaks[0])  # This is the array of masses.
print(peaks[1])  # This is the array of intensity values

q = spectrum.getInstrumentSettings()

[  204.99993902   205.00009595   205.00025288 ...,  2002.37720191
  2002.38199264  2002.38678339]
[ 0.  0.  0. ...,  0.  0.  0.]


In most cases, you'll probably be more interested in working with a centroided spectrum rather than the raw instrument data. OpenMS contains some powerful algorithms for signal processing, including a function specifically designed to pick peaks in FTMS data. In this next section we'll generate a profile-mode spectrum from our spectrum object and highlight how OpenMS method parameters can be configured. 

In [21]:
from pyopenms import PeakPickerHiRes

peakPicker = PeakPickerHiRes()

peakPicker.getParameters().asDict()

{b'SignalToNoise:auto_max_percentile': 95,
 b'SignalToNoise:auto_max_stdev_factor': 3.0,
 b'SignalToNoise:auto_mode': 0,
 b'SignalToNoise:bin_count': 30,
 b'SignalToNoise:max_intensity': -1,
 b'SignalToNoise:min_required_elements': 10,
 b'SignalToNoise:noise_for_empty_window': 1e+20,
 b'SignalToNoise:win_len': 200.0,
 b'SignalToNoise:write_log_messages': b'true',
 b'missing': 1,
 b'ms_levels': [1],
 b'report_FWHM': b'false',
 b'report_FWHM_unit': b'relative',
 b'signal_to_noise': 0.0,
 b'spacing_difference': 1.5,
 b'spacing_difference_gap': 4.0}

We've instantiated a PeakPickerHiRes object and printed out its parameters as a dictionary. It looks like right now the peak picker is set to only pick MS1 spectra, since the <code>ms_levels</code> parameter is set to <code>[1]</code>. Let's change it so that it will process both MS1 and MS2 spectra. The first step is going to be to create a new parameter object.  

In [22]:
from pyopenms import Param

peakPickerParams = Param()

peakPickerParams.setValue(b'ms_levels', [1, 2], '')

Notice that we've passed an empty string as the third argument in order to make the debugger happy. We also have to pass the parameter name, 'ms_levels,' as a bytestring by putting the <code>b</code> in front. Now that we have our param object, let's apply it to <code>peakPicker</code>:

In [23]:
peakPicker.setParameters(peakPickerParams)

peakPicker.getParameters().asDict()[b'ms_levels']

[1, 2]

Now our peak picker is properly configured to process MS1 and MS2 spectra. This basic pattern of instantiating a parameter object and applying it to a method is how settings are changed for any method in the pyOpenMS library. 

The next step is to actually apply our peak picker in order to process the spectrum. There are two ways to do this: we could either process individual spectra, or apply the algorithm to every scan in the experiment. First let's process the <code>spectrum</code> we defined earlier. The peak picker works by processing one <code>MSSpectrum</code> object and writing to a second <code>MSSpectrum</code>

In [24]:
centroidSpectrum = MSSpectrum()

peakPicker.pick(spectrum, centroidSpectrum) 

peaksinProfileSpectrum = spectrum.get_peaks()[0].size
peaksInCentroidSpectrum = centroidSpectrum.get_peaks()[0].size

print("The profile spectrum contains %d data points."%peaksinProfileSpectrum)

print("Our peak picker found %d peaks."%peaksInCentroidSpectrum)

The profile spectrum contains 198120 data points.
Our peak picker found 15500 peaks.


If your experiment contains multiple spectra, you can also apply the peakPicker to the entire experiment. Again, we create a new <code>MSExperiment</code> object to write our peaks into:

In [25]:
centroidExperiment = MSExperiment()

peakPicker.pickExperiment(exp, centroidExperiment)

# Here we're looking at the number of data poins in the first spectrum to make sure our peak picking worked. 
peaksinProfileExperiment = exp[0].get_peaks()[0].size
peaksInCentroidExperiment = centroidExperiment[0].get_peaks()[0].size

print("The profile spectrum contains %d data points."%peaksinProfileExperiment)

print("Our peak picker found %d peaks."%peaksInCentroidExperiment)

The profile spectrum contains 198120 data points.
Our peak picker found 15500 peaks.


The last basic operation on spectrum objects that I'm going to cover is how to write a processed spectra to a file. This is useful if you want to open up your spectrum in a MzML viewer such as TOPP or mMass. To write a file, we instantiate a new <code>MzMLFile</code> object and pass an <code>MSExperiment</code> to its <code>store</code> method. Alternatively, if we want to write an individual spectrum, we need to create a new <code>MSExperiment</code> and add our spectrum to it before writing to the file. The following chunk of code demonstrates both methods.  

In [26]:
# Method 1: Storing an experiment to a file
fileToExport = MzMLFile()
fileToExport.store('ProcessedExperiment.mzML', centroidExperiment)

# Method 2: Adding a spectrum to a new experiment, then writing to a file. 
experimentToWrite = MSExperiment()
experimentToWrite.addSpectrum(centroidSpectrum) 
fileToExport2 = MzMLFile()
fileToExport2.store('ProcessedSpectrum.mzML', experimentToWrite)

If you've read this far, you now know how to open a mass spectral file using pyOpenMS and perform a peak-picking analysis on both individual spectra and the full experiment. The library contains a lot of other useful processing methods, but the workflow for using them is going to be very similar: instantiate the method, provide it with a parameter object if necessary, and run the analysis on your data.

<h1>The Chemistry Module</h1>

The next section will cover some useful features of PyOpenMS's <code>Chemistry</code> module. The chemistry module contains a lot of neat methods that let you manipulate amino acid sequences to compute their masses, theoretical fragmentation, and isotope distributions. The following chunk of code shows how to create a sequence object for the peptide MASSSPECISNEAT and get some useful information out of it:

In [30]:
from pyopenms import AASequence

# generate a new sequence object from a string containing the sequence. 
myPeptideSequence = AASequence().fromString("MASSSPECISNEAT", True)

# Get the monoisotopic weight (for average weight, use getAverageWeight instead of getMonoWeight)
monoIsotopicWeightUncharged = myPeptideSequence.getMonoWeight(0, 0)
monoIsotopicWeight_3plus = myPeptideSequence.getMonoWeight(0, 3)

# Compute the molecular composition
formula_unprotonated = myPeptideSequence.getFormula(0, 0).toString()

# get the first 5 residues- positional arguments are start and endpoints
first5Residues = myPeptideSequence.getSubsequence(0, 5).toString()

You might have noticed that the functions to compute the weight and composition of the peptide take two positional arguments- e.g. <code>getFormula(0, 0)</code>. The first argument is a ResidueType code and determines whether the sequence should be treated as an intact peptide or as a fragment, according to the following lookup table:
<table>
    <tr><th>Key</th><th>Label</th><th>Description (From OpenMS documentation)</th></tr>
    <tr><th>0</th><th>Full</th><th>With N-terminus and C-terminus</th></tr>
    <tr><th>1</th><th>Internal</th><th>Without Termini</th></tr>
    <tr><th>2</th><th>N-terminal</th><th>Only N-terminus</th></tr>
    <tr><th>3</th><th>C-terminal</th><th>Only C-terminus</th></tr>
    <tr><th>4</th><th>A-Ion</th><th>N-terminus up to C-alpha/carbonyl carbon bond</th></tr>
    <tr><th>5</th><th>B-Ion</th><th>N-terminus up to the peptide bond</th></tr>
    <tr><th>6</th><th>C-Ion</th><th>N-terminus up to the amide/c-alpha bond</th></tr>
    <tr><th>7</th><th>X-Ion</th><th>Amide/c-alpha bond up to the C-terminus</tr>
    <tr><th>8</th><th>Y-Ion</th><th>Peptide bond up to the C-terminus</tr>
    <tr><th>9</th><th>Z-Ion</th><th>C-alpha/carbonyl carbon bond up to C-terminus</tr>
</table>

The second argument is the charge, and determines how many protons should be added to the sequence before computation. As a concrete example, calling <code>myPeptideaSequence.getMonoWeight(4, 3)</code> would return the monoisotopic mass of the a<sub>14</sub><sup>3+</sup> ion for a peptide with MASSSPECISNEAT as the first 14 residues.

<h2>Modification handling</h2>

PyOpenMS has Unimod integration, and you can add modifications using the SetModification command. Unfortunately modification that aren't in Unimod can't be added in this way.

In [31]:
myPeptideSequence.setModification(0, "Oxidation")

formula_oxidized = myPeptideSequence.getFormula(0, 0).toString()

print("The composition of the unoxidized peptide is %s"%formula_unprotonated)

print("The composition of the oxidized peptide is %s"%formula_oxidized)

The composition of the unoxidized peptide is b'C55H91N15O25S2'
The composition of the oxidized peptide is b'C55H91N15O26S2'


You can see that setting the odixation caused the formula to change to reflect the additional oxygen. 

PyOpenMS contains functions to compute the isotopic distributins of sequences and compositions:

In [32]:
from pyopenms import EmpiricalFormula

# compute the first 10 isotopes
myPeptide_isotopes = myPeptideSequence.getFormula(0, 0).getIsotopeDistribution(10)

# this getContainer function returns the isotope distribution as tuples of formula mass and abundance. 
myPeptide_isotopes.getContainer()


[(1441, 0.4383429723511864),
 (1442, 0.30098864515812396),
 (1443, 0.164891991080421),
 (1444, 0.06560323788654057),
 (1445, 0.021944548182978953),
 (1446, 0.006238062479179644),
 (1447, 0.0015597236074226704),
 (1448, 0.0003477242543181491),
 (1449, 7.01571171361993e-05),
 (1450, 1.293788269227358e-05)]

The EmpiricalFormula class can let you calculate isotopes distributions for arbitrary formulas. In the next example we'll add a bromine to the formula and compute the new isotope distribution. 

In [33]:
myComposition_brominated = myPeptideSequence.getFormula(0, 0).toString() + b'Br'

formulaObject = EmpiricalFormula(myComposition_brominated)

brominatedDistribution = formulaObject.getIsotopeDistribution(10)

brominatedDistribution.getContainer()

[(1520, 0.2222051573510236),
 (1521, 0.1525773959589096),
 (1522, 0.29974295165602416),
 (1523, 0.18167922636301667),
 (1524, 0.09243571975023783),
 (1525, 0.035512485566261465),
 (1526, 0.011611956395607774),
 (1527, 0.0032523832966202416),
 (1528, 0.0008046953251989233),
 (1529, 0.00017802833709978724)]

I hope that I've given you a good idea of what basic features OpenMS is capable of and that you're starting to have some ideas about how to harness this power for your own research. OpenMS has hundreds of classes and functions and I'll try to keep this SOP updated with the most useful ones. 